In [32]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [34]:
jishu1=250
jishu2=250

In [35]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: Ans: A or B",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [36]:
def get_bias_output(fact):
    template="""Please help me classify the following economic phenomenon.

Economic Phenomenon: %s

A. expansionary economic phenomenon   B. contractionary economic phenomenon"""%(fact)
    return template

In [37]:
now=0
data = pd.read_excel('./data/test1.xlsx', sheet_name=0)

In [38]:
while jishu1!=0 or jishu2!=0:
    if jishu1>0 and str(data['label'][now])=='0':

        ans=get_output(get_bias_output(data['sentence'][now]))
        try:
            ans=re.findall(r"Ans([\S|\s]*)",ans, re.S|re.I)[0]
        except:
            a=1
        if ("expansionary" in ans or "Expansionary" in ans or 'A' in ans) and 'B' not in ans and 'Neither' not in ans:
            jishu1=jishu1-1
            with open("./data/test.jsonl","a",encoding='utf-8') as k:
                input_dict={'number':now,'question':data['sentence'][now],'ans':str(data['label'][now])}
                input_json=json.dumps(input_dict)
                k.write(input_json+'\n')
    elif jishu2>0 and str(data['label'][now])=='1':
        ans=get_output(get_bias_output(data['sentence'][now]))
        try:
            ans=re.findall(r"Ans([\S|\s]*)",ans, re.S|re.I)[0]
        except:
            a=1
        #print(ans)
        if ("contractionary" in ans or "Contractionary" in ans or 'B' in ans) and 'A' not in ans and 'Neither' not in ans:
            jishu2=jishu2-1
            with open("./data/test.jsonl","a",encoding='utf-8') as k:
                input_dict={'number':now,'question':data['sentence'][now],'ans':str(data['label'][now])}
                input_json=json.dumps(input_dict)
                k.write(input_json+'\n')
    now=now+1

In [39]:
print(jishu1,jishu2)

0 0
